In [8]:

# import tools from WBT module

import sys
sys.path.insert(1, '/Users/madeleinegallop/Documents/ConnPlanning/wbt_starter')     # path points to my WBT directory

from WBT.whitebox_tools import WhiteboxTools

# declare a name for the tools

wbt = WhiteboxTools()


In [9]:
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  Working directories
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Define personal storage root.
# This is the path where you will store inputs and outputs from this workflow.
# For example, my root points to the directory (folder) of s23 in GEOG0310 
# on an external drive named drosera. 

root = "/Users/madeleinegallop/Documents/ConnPlanning"

# Set up separate directories to store temporary and keeper outputs. //writing to harddrive each time

temps = root+"/temps/"     
keeps = root+"/keeps/"   

In [3]:

#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  Required datasets:
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#we are trying to identify forested habitat blocks. start with the lchp (from GEE)
# Point to directory where you hold input data. 
# The 'midd' DEM is relatively small and good for testing. 

lc = root+"/inputs/LCHP_1m_midd.tif"  # includes trees, excludes non native veg


In [4]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# IMPLEMENT
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# ------------------------------------------------------------------------------
# 1. Identify clumps of recovering habitat. 
# ------------------------------------------------------------------------------

# Resample LCHP to 3 meter to speed up pilot. have to kick down the resolution, resample up from 1 or .5 up to 3

wbt.resample(
    inputs = lc, 
    output = temps+"_0301_resample.tif", 
    cell_size = 3, #resample
    base = None,  #if you have a mama raster (w same cell size), it will grab it from there. 
    method = "nn" #have 3 choices, makes sense to do this to preserve nominal data (values coming out match those going in) 
)

# Make a binary where recovering habitat = 0 and not recovering habitat = 1.

wbt.reclass(
    i = temps+"_0301_resample.tif", 
    output = temps+"_0302_binary.tif", 
    reclass_vals = "0;1;0;2;0;3;1;4;1;5;1;6;1;7;1;8;1;9;1;10", # a little wierd becuase of these. going from old value to new value
   #its in the form: i want 0 if its 1, i want 0 if its 2, i want 1 if its 4, etc
    assign_mode=True #otherwise i would have to assign ranges. 
) #now anything that is tree canopy and grass shrub are going to 0, everything else can go to 1. 
#now with the buffer, the buffer can go IN 

# Pass a majority filter over the binary to remove noise from random pixels.  
#smoother
wbt.majority_filter(
    i = temps+"_0302_binary.tif", 
    output = temps+"_0303_majority_filter.tif", 
    filterx=5, 
    filtery=5
)
#buffer into the natural habitat 
# Buffer 50 meters into recovering habitat to identify cores.  
# Because images sometimes lose their crs information, it can be safer to specific distance in cell units. 

wbt.buffer_raster( #buffer inwards
    i = temps+"_0303_majority_filter.tif", 
    output = temps+"_0304_buffer.tif", 
    size = 17, #our scale is 3 meters, so this is 51 meters. 
    gridcells=True #using number of pixels and not area on the map 
)

# Invert figure and ground.  

wbt.equal_to(
    input1 = temps+"_0304_buffer.tif", 
    input2 = 0, 
    output = temps+"_0305_invert.tif"
)

# Identify clumps of core recovering habitat. 
 
wbt.clump(
    i = temps+"_0305_invert.tif", 
    output = temps+"_0306_habitat_clumps.tif", 
    diag=True, 
    zero_back=True, #selfMask()
)
#now have clumps of grass, water and tree canopy (natural land cover). so now we want jsut the forested ones. 
# ------------------------------------------------------------------------------
# 2. Identify recovering clumps with a majority of tree canopy. 
# ------------------------------------------------------------------------------

# Compute area of each core recovering habitat clump.

wbt.raster_area(
    i = temps+"_0306_habitat_clumps.tif", 
    output = temps+"_0311_habitat_area.tif", 
    out_text=False, 
    units="grid cells",  #small area, so its ok to use grid cells. if we were doing state of vermont, area represeted by each pixel is gonna change, so that would skeew results. ok for small area. 
    zero_back=True
)

# Make a binary of tree canopy. 

wbt.reclass(
    i = temps+"_0301_resample.tif", 
    output = temps+"_0312_tree_canopy_binary.tif", 
    reclass_vals = "1;1;0;2;0;3;0;4;0;5;0;6;0;7;0;8;0;9;0;10", 
    assign_mode=True
)

# Count the number of tree canopy pixels in each clump.  

wbt.zonal_statistics(
    i = temps+"_0312_tree_canopy_binary.tif", #dough
    features = temps+"_0306_habitat_clumps.tif", #cutter 
    output = temps+"_0313_zonal_stats.tif", 
    stat="total", 
    out_table=None, 
)#gives area (# pixels) of tree canopy in each region. divide by area (#pixels) of each region. 
#so here, each pixel stores the # of pixels of tree binary that exist in the regions dictated in the cutters. 

# Divide the count of tree canopy by the pixel area of each clump. to get percent of forest cover in each clump. 

wbt.divide(
    input1 = temps+"_0313_zonal_stats.tif", 
    input2 = temps+"_0311_habitat_area.tif", 
    output = temps+"_0314_habitat_percent_tree.tif",
)

# Replace background value with zero.
#for when the division yields 1/0, plugs in 0 
wbt.convert_nodata_to_zero(
    i = temps+"_0314_habitat_percent_tree.tif", 
    output = temps+"_0315_habitat_percent_tree_bg0.tif", 
)

# Threshold habitat blocks by percent tree canopy (> 49%). 

wbt.greater_than(
    input1 = temps+"_0315_habitat_percent_tree_bg0.tif", 
    input2 = 0.49,  #anything greater than 49%. 
    output = temps+"_0316_habitat_majority_tree.tif", 
    incl_equals=False,
)

# ------------------------------------------------------------------------------
# 3. Identify clumps of tree canopy that overlap blocks          . 
# ------------------------------------------------------------------------------
#now we have core areas that are 50 m from edge and 50% tree canopy. so sometimes tree clumps intersect with the buffer zone, so have to specify this to prevent it. 
# Clump tree canopy binary to identify stands of trees. 
#buffering in makes it so it has to be a certain size, have a certain interior. now that we have filtered out the little clumps, we are building them back out a little 

wbt.clump(
    i = temps+"_0312_tree_canopy_binary.tif", 
    output = temps+"_0331_tree_canopy_clumps.tif", 
    diag=True, 
    zero_back=True, 
)

# Set background as no data. 

wbt.set_nodata_value(
    i = temps+"_0331_tree_canopy_clumps.tif", 
    output = temps+"_0333_tree_canopy_clumps_bg_nd.tif", 
    back_value=0.0, 
)


# Identify tree stands that overlap habitat blocks with majority trees. 
#this one does true or false does it overlap. max from the binary, grouped by the cutter

wbt.zonal_statistics(
    i = temps+"_0316_habitat_majority_tree.tif", 
    features = temps+"_0333_tree_canopy_clumps_bg_nd.tif",  #cutter is the clumps
    output = temps+"_0334_max_habitat_tree.tif", 
    stat="maximum", 
    out_table=None, 
) 

# ------------------------------------------------------------------------------
# 4. Combine habitat blocks with majority of trees and overlapping tree stands.         . 
# ------------------------------------------------------------------------------

# Replace background value with zeros for overlapping tree stands.

wbt.convert_nodata_to_zero(
    i = temps+"_0334_max_habitat_tree.tif", 
    output = temps+"_0341_max_habitat_tree_bg0.tif", 
)

# Union the overlapping tree stands and habitat blocks. 
#true or false do these objects overlap this zone
#get the tree canopies and smush them in. filling in non treecanpoy grass and water. 
wbt.Or(
    input1 = temps+"_0316_habitat_majority_tree.tif",  #objects
    input2 = temps+"_0341_max_habitat_tree_bg0.tif",  #boolean
    output = temps+"_0342_union.tif", 
)


# Identify individual forested habitat blocks. 

wbt.clump(
    i = temps+"_0342_union.tif", 
    output = temps+"_0343_forested_habitat_blocks.tif", 
    diag=True, 
    zero_back=True
)

# ------------------------------------------------------------------------------
# 5. Classify forested habitat blocks by area. 
# ------------------------------------------------------------------------------

# Set background as no data. 

wbt.set_nodata_value( #filling these in with 0
    i = temps+"_0343_forested_habitat_blocks.tif", 
    output = temps+"_0351_forested_habitat_blocks_bg_nd.tif", 
    back_value=0.0, 
)

# Compute the area of each block. 

wbt.raster_area(
    i = temps+"_0351_forested_habitat_blocks_bg_nd.tif", 
    output = temps+"_0352_forested_habitat_blocks_area.tif", 
    out_text=False, 
    units="map units", 
    zero_back=True
)

# Convert square meters to acres.

wbt.divide(
    input1 = temps+"_0352_forested_habitat_blocks_area.tif", 
    input2 = 4046.86,  #convert to acres 
    output = keeps+"_0353_forested_habitat_blocks_acres.tif",
)

#objects are considered to be zones. we can set the no data value if we want or convert 0 to no data. 
#there are times we want 0s like to do distance and times when we want the no data, like say we are taking 
#there has to be an egg in every slot in the egg holder
#0s are good for buffers, but sometimes we wanna mask 
#think 0 if you want distance 
#zonal sat max is like select by location 
#can use max to identify greatest disturbance calss 
#zonal stats provides summary stats for one layer b

#find valley bottoms that are not forest blocks that connect to habitat blocks. 71: twons are encouraged to ID connectivity regions in towns. 
#so now we have some habitat blocks, so the next thing is to see how they are connected. lets consider valley bottoms as bridges, get bridges vs piers vs rafts. 
#so start with all the valley bottoms, we dont really care about forest blocks. want how the valley bottoms connect the forested areas. 
#most important is water, which is in the lowlands




./whitebox_tools --run="Resample" --inputs='/Users/madeleinegallop/Documents/ConnPlanning/inputs/LCHP_1m_midd.tif' --output='/Users/madeleinegallop/Documents/ConnPlanning/temps/_0301_resample.tif' --cell_size='3' --method=nn -v --compress_rasters=False

****************************
* Welcome to Resample      *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
Progress: 36%
Progress: 37%
Progress: 38%
Progress: 39%
Progress: 40%
Progress: 41%
Progres

0